In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from src.databases import Postgresql 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFECV 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

In [3]:
# trazer o dataset com as viariaveis dummy do banco de dados
query = """ SELECT * FROM dataset_final;"""
bd = Postgresql(user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
dataset= bd.retrieve_data(query=query)
dataset.drop_duplicates(inplace=True)
dataset.set_index('sku', inplace=True)
dataset = dataset.astype('int')

Buscando os dados!!!
Conexão com Postgresql fechada


In [4]:
dataset.head()

,tipo,country,region,vinicola,classificacao,safra,temperatura,potencial_guarda,cat_safra,teor_cat,...,inox,metodo_tradicional,garrafas,concreto,gustativo,visual,ofativo,comments_label,uva,rating
sku,,,,,,,,,,,,,,,,,,,,,
11549,0,9,96,116,3,1,2,1,0,1,...,0,0,0,0,0,0,4,0,35,0
12390,0,8,45,10,2,16,2,0,0,1,...,1,0,0,0,0,0,4,0,85,0
12733,4,6,81,94,2,15,0,0,0,0,...,0,0,0,0,5,2,4,0,95,0
12734,0,4,102,37,2,17,1,0,0,2,...,1,0,0,0,0,0,4,0,24,0
12813,4,8,76,44,2,8,0,1,0,0,...,0,0,0,0,5,2,2,0,58,0


In [5]:
dataset.shape

(453, 21)

In [66]:
#divisão dos dados
X= dataset.drop('rating', axis=1)
y= dataset['rating']

#Divisão entre treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X,y,test_size = 0.2, random_state=104)

# Balanciando o dataset 
x, y = SMOTE().fit_resample(X_treino, y_treino)
print('Balanciamento realizado: ', sorted(Counter(y).items()))

Balanciamento realizado:  [(0, 334), (1, 334)]


In [53]:
x.shape

(668, 20)

In [54]:
X_teste.shape

(91, 20)

# Chi2/f_classif

In [55]:
# Testando chi2 ou f_classif

def chi2_anova(tec1, tec2):
    tecnicas = [tec1, tec2]

    for item in tecnicas:
        model_select = make_pipeline(
                    SelectKBest(item, k=5), MinMaxScaler(), LogisticRegression()
            )
        model_select.fit(x, y)
        print('Acuracia com {}: {:.3f}'.format(item, model_select.score(X_teste, y_teste)))

chi2_anova(chi2, f_classif)

Acuracia com <function chi2 at 0x7f5b4c6cfdc0>: 0.648
Acuracia com <function f_classif at 0x7f5b4c6cfa60>: 0.615


# RFE

In [57]:
#Regressão Logistica
rounds = [1,2,3]
acc = []
for r in rounds:
    clf = RandomForestClassifier()
    selector = RFECV(estimator=clf, cv=5).fit(x,y)
    index = selector.get_support(indices=True)
    x_ = x.iloc[:,index]
    X_teste_ =  X_teste.iloc[:,index]

    model_select = make_pipeline(
        MinMaxScaler(), ExtraTreesClassifier()
    )

    model_select.fit(x_, y)
    p=model_select.predict(X_teste_)
    acc.append(roc_auc_score(p, y_teste))
    print(f'Acuracia de : {roc_auc_score(p, y_teste)}')
print(f'A media da acuracia de cada round é {np.mean(acc)}')

Acuracia de : 0.4642857142857143
Acuracia de : 0.463855421686747
Acuracia de : 0.463855421686747
A media da acuracia de cada round é 0.46399885255306944


# Foward Elimination

In [59]:
# Testando quantidade de k - Foward Elimination
#Regresão Logistica

def n_k_rl(method, ks, model):
    num_k= ks
    for item in num_k:

        model_select = make_pipeline(
                SelectKBest(method, k=item), MinMaxScaler(),model()
        )
        model_select.fit(x, y)
        print('Acuracia com a quantidade k de {}: {:.3f}'.format(item, model_select.score(X_teste, y_teste)))

n_k_rl(chi2, ks=list(range(1,21)), model=LogisticRegression)

Acuracia com a quantidade k de 1: 0.505
Acuracia com a quantidade k de 2: 0.516
Acuracia com a quantidade k de 3: 0.451
Acuracia com a quantidade k de 4: 0.516
Acuracia com a quantidade k de 5: 0.648
Acuracia com a quantidade k de 6: 0.659
Acuracia com a quantidade k de 7: 0.659
Acuracia com a quantidade k de 8: 0.637
Acuracia com a quantidade k de 9: 0.670
Acuracia com a quantidade k de 10: 0.703
Acuracia com a quantidade k de 11: 0.703
Acuracia com a quantidade k de 12: 0.692
Acuracia com a quantidade k de 13: 0.714
Acuracia com a quantidade k de 14: 0.714
Acuracia com a quantidade k de 15: 0.714
Acuracia com a quantidade k de 16: 0.714
Acuracia com a quantidade k de 17: 0.736
Acuracia com a quantidade k de 18: 0.725
Acuracia com a quantidade k de 19: 0.725
Acuracia com a quantidade k de 20: 0.725


In [60]:
#ExtraTreesClassifier
def n_k_et(method, ks, model):
    num_k= ks
    for item in num_k:

        model_select = make_pipeline(
                SelectKBest(method, k=item), MinMaxScaler(),model()
        )
        model_select.fit(x, y)
        print('Acuracia com a quantidade k de {}: {:.3f}'.format(item, model_select.score(X_teste, y_teste)))

n_k_et(chi2, ks=list(range(1,21)), model=ExtraTreesClassifier)

Acuracia com a quantidade k de 1: 0.769
Acuracia com a quantidade k de 2: 0.813
Acuracia com a quantidade k de 3: 0.813
Acuracia com a quantidade k de 4: 0.802
Acuracia com a quantidade k de 5: 0.769
Acuracia com a quantidade k de 6: 0.758
Acuracia com a quantidade k de 7: 0.791
Acuracia com a quantidade k de 8: 0.813
Acuracia com a quantidade k de 9: 0.802
Acuracia com a quantidade k de 10: 0.857
Acuracia com a quantidade k de 11: 0.868
Acuracia com a quantidade k de 12: 0.824
Acuracia com a quantidade k de 13: 0.813
Acuracia com a quantidade k de 14: 0.846
Acuracia com a quantidade k de 15: 0.857
Acuracia com a quantidade k de 16: 0.835
Acuracia com a quantidade k de 17: 0.835
Acuracia com a quantidade k de 18: 0.857
Acuracia com a quantidade k de 19: 0.846
Acuracia com a quantidade k de 20: 0.857


In [61]:
#SVC

def n_k_svc(method, ks, model):
    num_k= ks
    for item in num_k:

        model_select = make_pipeline(
                SelectKBest(method, k=item),MinMaxScaler(),model(random_state=0)
        )
        model_select.fit(x, y)
        print('Acuracia com a quantidade k de {}: {:.3f}'.format(item, model_select.score(X_teste, y_teste)))

n_k_svc(chi2, ks=list(range(1,21)), model = SVC)

Acuracia com a quantidade k de 1: 0.495
Acuracia com a quantidade k de 2: 0.330
Acuracia com a quantidade k de 3: 0.418
Acuracia com a quantidade k de 4: 0.560
Acuracia com a quantidade k de 5: 0.670
Acuracia com a quantidade k de 6: 0.692
Acuracia com a quantidade k de 7: 0.703
Acuracia com a quantidade k de 8: 0.824
Acuracia com a quantidade k de 9: 0.813
Acuracia com a quantidade k de 10: 0.846
Acuracia com a quantidade k de 11: 0.835
Acuracia com a quantidade k de 12: 0.835
Acuracia com a quantidade k de 13: 0.813
Acuracia com a quantidade k de 14: 0.868
Acuracia com a quantidade k de 15: 0.857
Acuracia com a quantidade k de 16: 0.857
Acuracia com a quantidade k de 17: 0.857
Acuracia com a quantidade k de 18: 0.857
Acuracia com a quantidade k de 19: 0.857
Acuracia com a quantidade k de 20: 0.835


In [62]:
#Random Forest

def n_k_rf(method, ks, model):
    num_k= ks
    for item in num_k:

        model_select = make_pipeline(
                SelectKBest(method, k=item),model(random_state=0)
        )
        model_select.fit(x, y)
        print('Acuracia com a quantidade k de {}: {:.3f}'.format(item, model_select.score(X_teste, y_teste)))

n_k_rf(chi2, ks=list(range(1,21)), model=RandomForestClassifier)

Acuracia com a quantidade k de 1: 0.725
Acuracia com a quantidade k de 2: 0.769
Acuracia com a quantidade k de 3: 0.780
Acuracia com a quantidade k de 4: 0.791
Acuracia com a quantidade k de 5: 0.725
Acuracia com a quantidade k de 6: 0.769
Acuracia com a quantidade k de 7: 0.802
Acuracia com a quantidade k de 8: 0.824
Acuracia com a quantidade k de 9: 0.802
Acuracia com a quantidade k de 10: 0.835
Acuracia com a quantidade k de 11: 0.868
Acuracia com a quantidade k de 12: 0.857
Acuracia com a quantidade k de 13: 0.846
Acuracia com a quantidade k de 14: 0.846
Acuracia com a quantidade k de 15: 0.868
Acuracia com a quantidade k de 16: 0.857
Acuracia com a quantidade k de 17: 0.857
Acuracia com a quantidade k de 18: 0.846
Acuracia com a quantidade k de 19: 0.868
Acuracia com a quantidade k de 20: 0.857


In [63]:
k_best = SelectKBest(score_func=f_classif, k=16)
selected  = k_best.fit(X_treino,y_treino)
index = selected.get_support(indices=True)

print(f'Indices da variavies são : {index}')

#features
dados = dataset.iloc[:,index]

print('\n')
dados.head()

Indices da variavies são : [ 0  1  3  4  5  6  7  9 11 12 13 14 15 16 17 19]




,tipo,country,vinicola,classificacao,safra,temperatura,potencial_guarda,teor_cat,inox,metodo_tradicional,garrafas,concreto,gustativo,visual,ofativo,uva
sku,,,,,,,,,,,,,,,,
11549,0,9,116,3,1,2,1,1,0,0,0,0,0,0,4,35
12390,0,8,10,2,16,2,0,1,1,0,0,0,0,0,4,85
12733,4,6,94,2,15,0,0,0,0,0,0,0,5,2,4,95
12734,0,4,37,2,17,1,0,2,1,0,0,0,0,0,4,24
12813,4,8,44,2,8,0,1,0,0,0,0,0,5,2,2,58


# Testando o efeito do PCA

In [64]:
#PCA
#dataset_pca = dataset.copy()
#X= dataset_pca.drop('rating', axis=1)
#y= dataset_pca['rating']

In [68]:
#Regressão Logistica 11 componentes
# Normalização
scl = MinMaxScaler()
scl_x = scl.fit_transform(x)
print("Dados Normalizados")

for num in list(range(1,21)):
    
    # Redução da Dimensionalidade
    pca = PCA(n_components=num)
    x_fit = pca.fit(scl_x)
    x_red = pca.fit_transform(scl_x)
    print("Redução de Dimensionalidade concluída")
    print(f'Variância capturada de {round(x_fit.explained_variance_ratio_.sum() * 100, 2)}')

    # Variáveis de Treino e Teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(x_red, y, test_size=0.2, random_state=42)
    print("Divisão entre dados de treino e teste concluída")
    
    #Cross Validação
    reg = LogisticRegression().fit(X_treino,y_treino)
    print(np.mean(cross_val_score(reg, X_teste, y_teste, cv=3)))
    print('\n')

Dados Normalizados
Redução de Dimensionalidade concluída
Variância capturada de 32.83
Divisão entre dados de treino e teste concluída
0.5745791245791246


Redução de Dimensionalidade concluída
Variância capturada de 43.73
Divisão entre dados de treino e teste concluída
0.6569023569023568


Redução de Dimensionalidade concluída
Variância capturada de 53.53
Divisão entre dados de treino e teste concluída
0.7393939393939394


Redução de Dimensionalidade concluída
Variância capturada de 62.02
Divisão entre dados de treino e teste concluída
0.7462962962962963


Redução de Dimensionalidade concluída
Variância capturada de 68.9
Divisão entre dados de treino e teste concluída
0.7462962962962963


Redução de Dimensionalidade concluída
Variância capturada de 75.23
Divisão entre dados de treino e teste concluída
0.7462962962962963


Redução de Dimensionalidade concluída
Variância capturada de 80.3
Divisão entre dados de treino e teste concluída
0.7462962962962963


Redução de Dimensionalidade con

In [69]:
#Extra Trees 8 componentes
# Normalização
scl = MinMaxScaler()
scl_x = scl.fit_transform(x)
print("Dados Normalizados")

for num in list(range(1,21)):
    # Redução da Dimensionalidade
    pca = PCA(n_components=num)
    x_fit = pca.fit(scl_x)
    x_red = pca.fit_transform(scl_x)
    print("Redução de Dimensionalidade concluída")
    print(f'Variância capturada de {round(x_fit.explained_variance_ratio_.sum() * 100, 2)}')

    # Variáveis de Treino e Teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(x_red, y, test_size=0.2, random_state=42)
    print("Divisão entre dados de treino e teste concluída")
    
    #Cross Validação
    reg = ExtraTreesClassifier().fit(X_treino,y_treino)
    print(np.mean(cross_val_score(reg, X_teste, y_teste, cv=3)))
    print('\n')

Dados Normalizados
Redução de Dimensionalidade concluída
Variância capturada de 32.83
Divisão entre dados de treino e teste concluída
0.664141414141414


Redução de Dimensionalidade concluída
Variância capturada de 43.73
Divisão entre dados de treino e teste concluída
0.709090909090909


Redução de Dimensionalidade concluída
Variância capturada de 53.53
Divisão entre dados de treino e teste concluída
0.8203703703703704


Redução de Dimensionalidade concluída
Variância capturada de 62.02
Divisão entre dados de treino e teste concluída
0.8203703703703704


Redução de Dimensionalidade concluída
Variância capturada de 68.9
Divisão entre dados de treino e teste concluída
0.842929292929293


Redução de Dimensionalidade concluída
Variância capturada de 75.23
Divisão entre dados de treino e teste concluída
0.857912457912458


Redução de Dimensionalidade concluída
Variância capturada de 80.3
Divisão entre dados de treino e teste concluída
0.8353535353535353


Redução de Dimensionalidade concluí

In [70]:
#svc 
# Normalização
scl = MinMaxScaler()
scl_x = scl.fit_transform(x)
print("Dados Normalizados")

for num in list(range(1,21)):
    # Redução da Dimensionalidade
    pca = PCA(n_components=num)
    x_fit = pca.fit(scl_x)
    x_red = pca.fit_transform(scl_x)
    print("Redução de Dimensionalidade concluída")
    print(f'Variância capturada de {round(x_fit.explained_variance_ratio_.sum() * 100, 2)}')

    # Variáveis de Treino e Teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(x_red, y, test_size=0.2, random_state=42)
    print("Divisão entre dados de treino e teste concluída")
    
    #Cross Validação
    reg = SVC().fit(X_treino,y_treino)
    print(np.mean(cross_val_score(reg, X_teste, y_teste, cv=3)))
    print('\n')

Dados Normalizados
Redução de Dimensionalidade concluída
Variância capturada de 32.83
Divisão entre dados de treino e teste concluída
0.6343434343434343


Redução de Dimensionalidade concluída
Variância capturada de 43.73
Divisão entre dados de treino e teste concluída
0.7018518518518518


Redução de Dimensionalidade concluída
Variância capturada de 53.53
Divisão entre dados de treino e teste concluída
0.7390572390572391


Redução de Dimensionalidade concluída
Variância capturada de 62.02
Divisão entre dados de treino e teste concluída
0.7836700336700337


Redução de Dimensionalidade concluída
Variância capturada de 68.9
Divisão entre dados de treino e teste concluída
0.7686868686868685


Redução de Dimensionalidade concluída
Variância capturada de 75.23
Divisão entre dados de treino e teste concluída
0.7609427609427609


Redução de Dimensionalidade concluída
Variância capturada de 80.3
Divisão entre dados de treino e teste concluída
0.7685185185185185


Redução de Dimensionalidade con

In [71]:
#Random Forest
# Normalização
scl = MinMaxScaler()
scl_x = scl.fit_transform(x)
print("Dados Normalizados")

for num in list(range(1,21)):
    # Redução da Dimensionalidade
    pca = PCA(n_components=num)
    x_fit = pca.fit(scl_x)
    x_red = pca.fit_transform(scl_x)
    print("Redução de Dimensionalidade concluída")
    print(f'Variância capturada de {round(x_fit.explained_variance_ratio_.sum() * 100, 2)}')

    # Variáveis de Treino e Teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(x_red, y, test_size=0.2, random_state=42)
    print("Divisão entre dados de treino e teste concluída")
    
    #Cross Validação
    reg = RandomForestClassifier().fit(X_treino,y_treino)
    print(np.mean(cross_val_score(reg, X_teste, y_teste, cv=3)))
    print('\n')

Dados Normalizados
Redução de Dimensionalidade concluída
Variância capturada de 32.83
Divisão entre dados de treino e teste concluída
0.664141414141414


Redução de Dimensionalidade concluída
Variância capturada de 43.73
Divisão entre dados de treino e teste concluída
0.7164983164983165


Redução de Dimensionalidade concluída
Variância capturada de 53.53
Divisão entre dados de treino e teste concluída
0.8506734006734007


Redução de Dimensionalidade concluída
Variância capturada de 62.02
Divisão entre dados de treino e teste concluída
0.812962962962963


Redução de Dimensionalidade concluída
Variância capturada de 68.9
Divisão entre dados de treino e teste concluída
0.842929292929293


Redução de Dimensionalidade concluída
Variância capturada de 75.23
Divisão entre dados de treino e teste concluída
0.8505050505050505


Redução de Dimensionalidade concluída
Variância capturada de 80.3
Divisão entre dados de treino e teste concluída
0.7981481481481482


Redução de Dimensionalidade conclu